# RNN - séries temporelles

Dans ce notebook, nous allons construire des RNNs de prédiction de séries temporelles univariées. 

Un premier dataset est le prix d'ouverture de cotation de l'action Google.
Nous implémentons 3 RNNs : simple, LSTM et GRU

Votre mission :
Modifier les architectures et paramètres des modeles pour estimer les performances, rapidité d'exécution et complexité des modèles.  
N'hésitez pas à ajouter tensorboard pour analyser les modèles.
Battre le score de `mean_absolute_error` de 1.78
Dans un second temps, appliquez ces modèles sur un autre dataset comme celui des taches solaires


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import math

pd.set_option('display.max_columns',30)
pd.set_option('display.max_rows',10)


In [ ]:
#the start and end date
start_date = dt.datetime(2020,4,1)
end_date = dt.datetime(2023,4,1)

#loading from yahoo finance
data = yf.download("GOOGL",start_date, end_date)

print()
print()

print(data.head())


In [ ]:
# Setting 80 percent data for training
training_data_len = math.ceil(len(data) * .8)
training_data_len

#Splitting the dataset
train_data = data[:training_data_len].iloc[:,:1]
test_data = data[training_data_len:].iloc[:,:1]
print(train_data.shape, test_data.shape)


In [ ]:
# Selecting Open Price values
dataset_train = train_data.Open.values
# Reshaping 1D to 2D array
# dataset_train = np.reshape(dataset_train, (-1,1))
# dataset_train.shape
dataset_train[:2]

In [ ]:
dataset_train = np.reshape(dataset_train, (-1,1))
dataset_train.shape

In [ ]:
dataset_train[:2]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_train = scaler.fit_transform(dataset_train)

print(scaled_train[:5])


In [ ]:
# Selecting Open Price values
dataset_test = test_data.Open.values
# Reshaping 1D to 2D array
dataset_test = np.reshape(dataset_test, (-1,1))
# Normalizing values between 0 and 1
scaled_test = scaler.fit_transform(dataset_test)
print(*scaled_test[:5])


In [ ]:
X_train = []
y_train = []
for i in range(50, len(scaled_train)):
	X_train.append(scaled_train[i-50:i, 0])
	y_train.append(scaled_train[i, 0])
	if i <= 51:
		print(X_train)
		print(y_train)
		print()


In [ ]:
X_test = []
y_test = []
for i in range(50, len(scaled_test)):
	X_test.append(scaled_test[i-50:i, 0])
	y_test.append(scaled_test[i, 0])


In this step, the data is converted into a format that is suitable for input to an RNN. np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) transforms the X_train array, which was originally a 2-dimensional array of shape (samples, features), into a 3-dimensional array of shape (samples, time steps, features), where time steps denotes the number of time steps in the input sequence and features denotes the number of features in the input data. Size 1 is an additional dimension that serves as an indication that each time step only has a single feature.

The y_train array is transformed from a 1-dimensional array of shape (samples) into a 2-dimensional array of shape (samples, 1) by np.reshape(y_train, (y_train.shape[0], 1)), where each row represents the output value at a certain time step.

In [ ]:
# The data is converted to Numpy array
X_train, y_train = np.array(X_train), np.array(y_train)

#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
y_train = np.reshape(y_train, (y_train.shape[0],1))
print("X_train :",X_train.shape,"y_train :",y_train.shape)


In [ ]:
# The data is converted to numpy array
X_test, y_test = np.array(X_test), np.array(y_test)

#Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
print("X_test :",X_test.shape,"y_test :",y_test.shape)


# Modelisation


In [ ]:
# importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import GRU, Bidirectional
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import mean_squared_error


ce modele defini un architecture bien particuliere

- RNN(50), Dropout, 3 RNN(50) et Dense pour la sortie

Que se passe t il si on change l'architecturer, si on la simplifie


In [ ]:
# initializing the RNN
regressor = Sequential()

# adding RNN layers and dropout regularization
regressor.add(SimpleRNN(units = 50,
						activation = "tanh",
						return_sequences = True,
						input_shape = (X_train.shape[1],1)))

regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units = 50,
						activation = "tanh",
						return_sequences = True))

regressor.add(SimpleRNN(units = 50,
						activation = "tanh",
						return_sequences = True))

regressor.add( SimpleRNN(units = 50))

# adding the output layer
regressor.add(Dense(units = 1,activation='sigmoid'))

# compiling RNN
regressor.compile(optimizer = SGD(learning_rate=0.01,
								momentum=0.9,
								nesterov=True),
				loss = "mean_squared_error")

# fitting the model
regressor.fit(X_train, y_train, epochs = 20, batch_size = 2)
regressor.summary()


## Evaluation


In [ ]:
y_RNN = regressor.predict(X_test)
y_RNN_O = scaler.inverse_transform(y_RNN)

In [ ]:
y_RNN_O[:3]

In [ ]:
y_test_O = scaler.inverse_transform(y_test)

In [ ]:
y_test_O[:3]

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test_O, y_RNN_O)


In [ ]:
fig, axs = plt.subplots(1,figsize =(12,6),sharex=True, sharey=True)
fig.suptitle('Model Predictions')

#Plot for RNN predictions
plt.plot(train_data.index[150:], train_data.Open[150:], label = "train_data", color = "b")
plt.plot(test_data.index, test_data.Open, label = "test_data", color = "g")
plt.plot(test_data.index[50:], y_RNN_O, label = "y_RNN", color = "black")
plt.grid()
plt.legend()
plt.title("Basic RNN")


plt.xlabel("Days")
plt.ylabel("Open price")

plt.show()


# LSTM



In [ ]:
#Initialising the model
regressorLSTM = Sequential()

#Adding LSTM layers
regressorLSTM.add(LSTM(50,
					return_sequences = True,
					input_shape = (X_train.shape[1],1)))
regressorLSTM.add(LSTM(50,
					return_sequences = False))
regressorLSTM.add(Dense(25))

#Adding the output layer
regressorLSTM.add(Dense(1))

#Compiling the model
regressorLSTM.compile(optimizer = 'adam',
					loss = 'mean_squared_error'
					)



In [ ]:
#Fitting the model
regressorLSTM.fit(X_train,
				y_train,
				batch_size = 1,
				epochs = 12)
regressorLSTM.summary()


In [ ]:
y_LSTM = regressorLSTM.predict(X_test)
y_LSTM_O = scaler.inverse_transform(y_LSTM)

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test_O, y_LSTM_O)

In [ ]:
fig, axs = plt.subplots(1,figsize =(12,6),sharex=True, sharey=True)
fig.suptitle('Model Predictions')

#Plot for LSTM predictions
plt.plot(train_data.index[150:], train_data.Open[150:], label = "train_data", color = "b")
plt.plot(test_data.index, test_data.Open, label = "test_data", color = "g")
plt.plot(test_data.index[50:], y_LSTM_O, label = "y_LSTM", color = "black")
plt.grid()
plt.legend()
plt.title("LSTM")


plt.xlabel("Days")
plt.ylabel("Open price")

plt.show()

# GRU

Implementer le meme modele avec une couche GRU

```
regressorGRU = Sequential()

# GRU layers with Dropout regularisation
regressorGRU.add(GRU(units=50,
                     return_sequences=True,
                     input_shape=(X_train.shape[1],1),
                     activation='tanh'))
```

# resultats

 quelle methode est meilleure ?

 en terme de
 - rapidité ?
 - complexité ?
 - performance ?


# sunspots

Dataset alternatif

url = https://raw.githubusercontent.com/SkatAI/skatai_deeplearning/master/data/sunspots.csv

Etapes :

- split train, test
- reshape en [[]]
- normaliser avec MinMaxScaler
- construire les sequences

```
url = "https://raw.githubusercontent.com/SkatAI/skatai_deeplearning/master/data/sunspots.csv"

spots = pd.read_csv(url)
```
